In [51]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np
from tensorflow import keras
import time
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures
import pandas as pd

In [58]:
dfTrainz = pd.read_csv('SVMData/train_feat.csv', index_col=0)
dfTestz = pd.read_csv('SVMData/test_feat.csv', index_col=0)
dfTrainz.shape, dfTestz.shape
dfTrainz.dtypes
dataset = pd.concat([dfTrainz,dfTestz])
dataset.shape, dfTrainz.shape, dfTestz.shape
dataset['gender'].unique()
dataset

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,truth,username,course_id,age,gender,education,user_enroll_num,course_enroll_num,cluster_label,course_category
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,5981,course-v1:TsinghuaX+70800232X+2015_T2,1.504677,0.621567,1.697428,-0.007581,-0.138544,2,0.0
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,1544995,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,-0.780080,-0.504540,-0.360012,-0.138544,2,0.0
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,1,1072798,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,-0.780080,-0.504540,-0.095689,-0.138544,2,0.0
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,0,561867,course-v1:TsinghuaX+70800232X+2015_T2,2.085269,2.023214,3.899395,-0.007581,-0.138544,2,0.0
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,1368125,course-v1:TsinghuaX+70800232X+2015_T2,-0.599969,-0.780080,-0.504540,0.432957,-0.138544,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393203,-0.225912,-0.225912,-0.243043,-0.296739,-0.321027,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,29394,course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS,1.432103,2.023214,1.697428,-0.007581,-0.695476,2,0.0
393204,-0.225912,-0.225912,-0.243043,-0.296739,-0.321027,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,1,3238122,course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS,-0.599969,-0.780080,-0.504540,-0.183797,-0.695476,2,0.0
131061,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,1,1103805,course-v1:TsinghuaX+20220053X_2015_T2+2015_T2,-0.599969,-0.780080,-0.504540,-0.271904,-0.740180,2,0.0


In [59]:
#dataset['course_id'] = pd.Categorical(dataset['course_id'])
#dataset['course_id'] = dataset['course_id'].cat.codes
#dataset['course_id'].unique()

target = dataset.pop('truth')
trash= dataset.pop('course_id')
trash2= dataset.pop('course_category')

trash3= dataset.pop('username')
dataset.head()
#prepdataset = tf.data.Dataset.from_tensor_slices((dataset.values, target.values))
#type(prepdataset)
#train_dataset = dataset.sample(frac = 0.8, random_state=0)
#test_dataset = dataset.drop(train_dataset.index)
#train_labels = train_dataset.pop('truth')
#test_labels = test_dataset.pop('truth')
#dataset.head()

,all#count,session#count,seek_video#num,play_video#num,pause_video#num,stop_video#num,load_video#num,problem_get#num,problem_check#num,problem_save#num,...,click_about#num,click_forum#num,click_progress#num,close_courseware#num,age,gender,education,user_enroll_num,course_enroll_num,cluster_label
enroll_id,,,,,,,,,,,,,,,,,,,,,
772,-0.228387,-0.228387,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,-0.293695,-0.17735,-0.284796,-0.464309,1.504677,0.621567,1.697428,-0.007581,-0.138544,2
773,-0.204883,-0.204883,-0.115253,-0.242027,-0.296012,-0.059298,-0.445267,-0.042412,-0.305844,-0.202161,...,-0.159877,-0.17735,-0.284796,-0.428065,-0.599969,-0.780080,-0.504540,-0.360012,-0.138544,2
774,-0.139320,-0.139320,-0.051358,-0.105248,-0.208460,-0.057709,-0.195358,-0.042412,-0.305844,-0.202161,...,-0.427513,-0.17735,-0.284796,-0.138111,-0.599969,-0.780080,-0.504540,-0.095689,-0.138544,2
776,-0.207357,-0.207357,-0.243043,-0.255705,-0.296012,-0.057709,-0.409566,-0.042412,-0.305844,-0.202161,...,-0.159877,-0.17735,-0.284796,-0.391821,2.085269,2.023214,3.899395,-0.007581,-0.138544,2
777,-0.225912,-0.225912,-0.243043,-0.296739,-0.333535,-0.059298,-0.480968,-0.042412,-0.305844,-0.202161,...,-0.293695,-0.17735,-0.284796,-0.428065,-0.599969,-0.780080,-0.504540,0.432957,-0.138544,3


In [54]:
dataset.shape

(225642, 29)

In [60]:
model = keras.Sequential(
    [
        keras.Input(shape=(None, 29)),
        RandomFourierFeatures(
            output_dim=4096, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=10),
    ]
)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)




ValueError: The rank of the input tensor should be 2. Got 3 instead.

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( dataset, target, test_size=0.33, random_state=42)
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5


ValueError: in user code:

    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\zunai\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

In [48]:

train_feat= pd.read_csv('Datasets/train_features.csv', index_col=0)
train_feat.head()
#train_feat.columns
cid = [197, 195, 107, 200, 101, 237, 165,  22, 194, 125, 231, 114, 198,
        12,   5, 161, 184, 205,  44,  42,  43, 225, 137, 106,  45, 120,
        61,   7, 147, 162,  62,  66,  13,  38, 135, 152, 128, 148, 154,
       182, 183, 102, 241,  46, 103, 179, 202,  89,  30, 133, 172,   6,
       111, 112,   0,  14, 130, 169,  31,  73,  36,  58,  74,  75, 170,
        23,  41,  28, 105, 159, 145,   9,  48,  29,  21, 243,  69, 233,
       213, 209,  79,  78,  80, 117,   8, 201, 140, 127, 149, 164, 232,
        49,  63, 116,  98, 109, 108, 115, 238, 189, 188,  24, 190, 143,
         4, 222,   3, 146,  67, 121, 199, 144, 155,  76, 157,  72, 166,
       203, 214,  60,  37, 163, 104, 175, 122, 177, 239,  70,  59, 216,
        92,  91, 139,  68, 153, 221,   1,  33,  96, 110,  27, 181, 167,
        57, 236, 235, 215, 192, 204,  26,  52, 206,  53, 193, 100, 191,
       168,  86, 138,  11, 224, 229, 228, 230, 186, 227, 223, 218,   2,
       158,  20,  82,  95,  94,  54,  51,  97, 220, 151, 187, 173, 174,
        84,  83,  85, 118, 211, 119, 212,  34,  88, 131, 132,  15, 226,
       180,  47, 171,  39, 242,  18, 156, 185, 123, 196,  99, 124, 240,
        77,  25,  32,  40, 217, 234,  64, 178, 176,  35,  71,  19, 113,
        81, 207, 208,  90,  10,  17, 160,  93, 246,  65,  50, 245,  87,
       134, 126, 136, 129, 142, 141, 150,  16, 210,  56,  55, 244, 219]

dataset()

TypeError: 'DataFrame' object is not callable

enroll_id
772       0.621567
773      -0.780080
774      -0.780080
776       2.023214
777      -0.780080
            ...   
393203    2.023214
393204   -0.780080
131061   -0.780080
131065    0.621567
393214   -0.780080
Name: gender, Length: 225642, dtype: float64

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:

train_truth = pd.read_csv('prediction_log/train_log.csv')
train_truth

In [1]:
y= dfTrainz.groupby('cluster_label')
y.head()

NameError: name 'dfTrainz' is not defined

In [64]:
courseinfo.head()

,id,course_id,start,end,course_type,category
0,6561,course-v1:CPVS+CPVS-HDLSC001+20160901,2016-11-16 08:00:00,2016-12-31 23:30:00,0,NaN
1,5557,course-v1:SCUT+144282+201709,2016-09-01 00:00:00,2017-02-28 00:00:00,0,NaN
2,9433,course-v1:ZK+06093+J,2018-01-01 08:00:00,2020-01-01 00:00:00,0,NaN
3,8320,course-v1:nuist+001+2016-T1,2017-03-01 18:30:00,2017-07-01 23:30:00,0,NaN
4,231,FUDAN/CFD004/2014.9-2015.1,2014-09-10 08:00:00,2015-09-10 00:00:00,0,NaN


In [65]:

def dataparse(train, test, num_feat, cat_feat):
    all_data = pd.concat([train, test])
    feat_dim = 0
    feat_dict = dict()
    for f in cat_feat:
        cat_val = all_data[f].unique()
        feat_dict[f] = dict(zip(cat_val, range(feat_dim, len(cat_val) + feat_dim)))
        feat_dim += len(cat_val)
    for f in num_feat:
        feat_dict[f] = feat_dim
        feat_dim += 1
    data_indice = all_data.copy()
    data_value = all_data.copy()
    for f in all_data.columns:
        if f in num_feat:
            data_indice[f] = feat_dict[f]
        elif f in cat_feat:
            data_indice[f] = data_indice[f].map(feat_dict[f])
            data_value[f] = 1.
        else:
            data_indice.drop(f, axis=1, inplace=True)
            data_value.drop(f, axis=1, inplace=True)
    return feat_dim, data_indice, data_value



In [66]:

def load_data():
    dfTrain = pd.read_csv('feat/train_feat.csv', index_col=0)
    dfTest = pd.read_csv('feat/test_feat.csv', index_col=0)
    dfTrain_u = dfTrain[user_num_feat + user_cat_feat + ['truth']]
    dfTest_u = dfTest[user_num_feat + user_cat_feat + ['truth']]

    dfTrain_c = dfTrain[course_num_feat + course_cat_feat + ['truth']]
    dfTest_c = dfTest[course_num_feat + course_cat_feat + ['truth']]
    dfTrain_a = dfTrain[act_feat_basic + ['truth', 'username', 'course_id']]
    dfTest_a = dfTest[act_feat_basic + ['truth', 'username', 'course_id']]

    return dfTrain_u, dfTest_u, dfTrain_c, dfTest_c, dfTrain_a, dfTest_a

In [67]:
def feat_augment(train_feat, test_feat):
    act_feats = pkl.load(open('act_feats.pkl', 'rb'))
    all_feat = pd.concat([train_feat, test_feat])
    all_feat_u_mean = all_feat.groupby('username').mean()[act_feats]
    all_feat_u_mean.columns = [x + '#user#mean' for x in all_feat_u_mean.columns]
    all_feat_u_max = all_feat.groupby('username').max()[act_feats]
    all_feat_u_max.columns = [x + '#user#max' for x in all_feat_u_max.columns]

    all_feat = pd.merge(all_feat, all_feat_u_mean, right_index=True, left_on='username')
    all_feat = pd.merge(all_feat, all_feat_u_max, right_index=True, left_on='username')

    all_feat_c_mean = all_feat.groupby('course_id').mean()[act_feats]
    all_feat_c_mean.columns = [x + '#course#mean' for x in all_feat_c_mean.columns]

    all_feat_c_max = all_feat.groupby('course_id').max()[act_feats]
    all_feat_c_max.columns = [x + '#course#max' for x in all_feat_c_max.columns]

    all_feat = pd.merge(all_feat, all_feat_c_mean, right_index=True, left_on='course_id')
    all_feat = pd.merge(all_feat, all_feat_c_max, right_index=True, left_on='course_id')

    act_feat = []
    for f in act_feat_basic:
        act_feat.append(f)
        act_feat.append(f + '#user#mean')
        act_feat.append(f + '#user#max')
        act_feat.append(f + '#course#mean')
        act_feat.append(f + '#course#max')

    return all_feat.loc[train_feat.index], all_feat.loc[test_feat.index], act_feat

In [6]:

    dfTrainzz = pd.read_csv('course_info.csv', index_col=0)
    dfTrainzz.head()

NameError: name 'pd' is not defined

In [69]:


# load data
dfTrain_u, dfTest_u, dfTrain_c, dfTest_c, dfTrain_a, dfTest_a = load_data()
dfTrain_a, dfTest_a, act_feat = feat_augment(dfTrain_a, dfTest_a)
dfTest_a['course_id']

enroll_id
131072        course-v1:TsinghuaX+20220053X_2015_T2+2015_T2
131073        course-v1:TsinghuaX+20220053X_2015_T2+2015_T2
393217    course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS
393221    course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS
131079        course-v1:TsinghuaX+20220053X_2015_T2+2015_T2
                                ...                        
393203    course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS
393204    course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS
131061        course-v1:TsinghuaX+20220053X_2015_T2+2015_T2
131065        course-v1:TsinghuaX+20220053X_2015_T2+2015_T2
393214    course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2016_TS
Name: course_id, Length: 67699, dtype: object

In [81]:
 u_feat_dim, u_data_indice, u_data_value = dataparse(dfTrain_u, dfTest_u, user_num_feat, user_cat_feat)
 u_feat_dim
ui_train, uv_train = np.asarray(u_data_indice.loc[dfTrain_u.index], dtype=int), np.asarray(
        u_data_value.loc[dfTrain_u.index], dtype=float)

av_train = np.asarray(dfTrain_a[act_feat], dtype=float)
dfTrain_a[act_feat].shape

(157943, 115)

In [35]:
    u_feat_dim, u_data_indice, u_data_value = dataparse(dfTrain_u, dfTest_u, user_num_feat, user_cat_feat)
    ui_train, uv_train = np.asarray(u_data_indice.loc[dfTrain_u.index], dtype=int), np.asarray(
        u_data_value.loc[dfTrain_u.index], dtype=float)
    ui_test, uv_test = np.asarray(u_data_indice.loc[dfTest_u.index], dtype=int), np.asarray(
        u_data_value.loc[dfTest_u.index], dtype=float)

    params["u_feat_size"] = u_feat_dim
    params["u_field_size"] = len(ui_train[0])

    c_feat_dim, c_data_indice, c_data_value = dataparse(dfTrain_c, dfTest_c, course_num_feat, course_cat_feat)
    ci_train, cv_train = np.asarray(c_data_indice.loc[dfTrain_c.index], dtype=int), np.asarray(
        c_data_value.loc[dfTrain_c.index], dtype=float)
    ci_test, cv_test = np.asarray(c_data_indice.loc[dfTest_c.index], dtype=int), np.asarray(
        c_data_value.loc[dfTest_c.index], dtype=float)

    params["c_feat_size"] = c_feat_dim
    params["c_field_size"] = len(ci_train[0])

    av_train = np.asarray(dfTrain_a[act_feat], dtype=float)

    ai_train = np.asarray([range(len(act_feat)) for x in range(len(dfTrain_a))], dtype=int)
    av_test = np.asarray(dfTest_a[act_feat], dtype=float)
    ai_test = np.asarray([range(len(act_feat)) for x in range(len(dfTest_a))], dtype=int)

    params["a_feat_size"] = len(ai_train[0])
    params["a_field_size"] = len(ai_train[0])

    y_train = np.asarray(dfTrain_a['truth'], dtype=int)
    y_test = np.asarray(dfTest_a['truth'], dtype=int)
    model = CFIN(**params)

    # generate valid set
    train_num = len(y_train)
    indices = np.arange(train_num)
    np.random.shuffle(indices)
    split_ = int(0.8 * train_num)
    ui_train_, ui_valid = ui_train[indices][:split_], ui_train[indices[split_:]]
    uv_train_, uv_valid = uv_train[indices][:split_], uv_train[indices[split_:]]
    ci_train_, ci_valid = ci_train[indices][:split_], ci_train[indices[split_:]]
    cv_train_, cv_valid = cv_train[indices][:split_], cv_train[indices[split_:]]
    ai_train_, ai_valid = ai_train[indices][:split_], ai_train[indices[split_:]]
    av_train_, av_valid = av_train[indices[:split_]], av_train[indices[split_:]]
    y_train_, y_valid = y_train[indices[:split_]], y_train[indices[split_:]]


NameError: name 'dfTrain_u' is not defined